In [ ]:
from google.colab import drive
drive.mount('/content/gdrive') #, force_remount=True

Mounted at /content/gdrive


In [ ]:
import os
os.chdir('gdrive/My Drive/Colab Notebooks/miramedix')

# import

In [ ]:
import re
import requests
import json

import xml.etree.ElementTree as ET


from bs4 import BeautifulSoup
from nltk import sent_tokenize

# load extracted terms (UMLS)

In [ ]:
import json

with open('terms_filtered_4.json', 'r') as f:
  annot_dict = json.load(f)

In [ ]:
for i in annot_dict.keys():
  print(i, len(annot_dict[i][0]))
  break

1 5


In [ ]:
for i in annot_dict['1'][0]:
  print(i['term'])

anaplastic astrocytomas
anaplastic astrocytoma
hypertension
tumor
thigh numbness


# Medline search

In [ ]:
def medlineplus_search_by_term(term):
    
    #retmax - максимальное число возвращаемых текстов
    url = 'https://wsearch.nlm.nih.gov/ws/query?db=healthTopics&term={}&retmax=3'.format(term) #5
    data = requests.get(url)
    string_xml = data.content
    tree = ET.fromstring(string_xml)
    titles = []
    urls = []
    texts = []

    for el in tree:
        if el.tag == 'list':
            for j in el:
                urls.append(j.attrib['url'] + '\n')
                for k in j:
                    if k.attrib['name'] == 'title':
                        titles.append('<h1>' + k.text + '</h1> \n')
                    elif k.attrib['name'] == 'FullSummary':
                        texts.append(k.text + '\n')
                        
    result = [x + y + z for x, y, z in zip(titles, urls, texts)]
    #ru_titles = [get_translation(x, 'en', 'ru') for x in titles]
    
    return result#, titles, texts, ru_titles

In [ ]:
medlineplus_search_by_term('hypertension')

['<h1><span class="qt0"><span class="qt1">High Blood Pressure</span></span></h1> \nhttps://medlineplus.gov/highbloodpressure.html\nWhat is blood pressure?<p>Blood pressure is the force of your blood pushing against the walls of your arteries. Each time your heart beats, it pumps blood into the arteries. Your <span class="qt0"><span class="qt1">blood pressure is highest</span></span> when your heart beats, pumping the blood. This is called systolic pressure. When your heart is at rest, between beats, your blood pressure falls. This is called diastolic pressure.</p><p>Your blood pressure reading uses these two numbers. Usually the systolic number comes before or above the diastolic number. For example, 120/80 means a systolic of 120 and a diastolic of 80.</p>How is <span class="qt0"><span class="qt1">high blood pressure</span></span> diagnosed?<p><span class="qt0"><span class="qt1">High blood pressure</span></span> usually has no symptoms. So the only way to find out if you have it is to

In [ ]:
stop_words = ['1-2', '1-2 times', '1 time per day', '2 times', '3 mm', '3 times', '4 times', '5 mm', 
              '6-10 times',
              'abdominal', 'Abreaction', 'absences', 'Activity', 'admission', 'advice', 'anesthesia', 
              'anesthesias', 'Anesthetic', 
              'anesthetic', 'Appearance', 'Application', 'application', 'applications', 'arm',
              'Asleep', 'asleep', 'asymptomatic', 'at admission', 'At home', 'attacks',
              'Bacterium', 'bleed', 'bleeding', 'bleeding time', 'Blood', 'blood', 'Bloody', 'burning',
              'burnings',
              'carried', 'Chronic', 'Center', 'Clearance', 'clinical diagnosis', 'Coagulation', 'coagulation',
              'complaints', 'condition', 'confirmed', 'consideration', 'Constriction', 'constriction', 
              'Consultation', 'consultation', 'consultations', 'Contact', 'contact',
              'day', 'decreas', 'decreased', 'determination', 'Development', 'development', 'diagnose', 
              'diagnosed', 'diagnosis', 'difficulty', 'disc', 'disappointment', 'Discharge',
              'discharge', 'Discharged', 'disease', 'diseases', 'disorders', 'drugs', 'dysfunction',
              'effect', 'elevation', 'emergency', 'Emotional', 'emotional', 'Erythrocytes', 'Examination', 
              'examination', 'examinations', 'Examined', 'examined', 'Exercise', 'exercises', 'extraction',
              'falls', 'field', 'first sign', 'first signs', 'Followed', 'follow-up', 'food',
              'gas', 'General', 'grade', 'gradex', 'groups',
              'Hemoglobin', 'hospitalized',
              'Improved', 'improved', 'Increased', 'injection', 'intervention', 'intolerance', 'Intolerance', 
              'irradiation', 'irritable',
              'Laboratory', 'lead increased', 'lesion', 'lesions', 'Leukocytes', 'Local', 'Low', 'low',
              'Medical', 'Medications', 'medications', 'Monitoring', 'monitoring', 'Multifocals',
              'Negative', 'negative', 'No development', 'noises', 'Nonspecific',
              'organ', 'organs', 'outpatient treatment',
              'Pain', 'pain', 'pains', 'palpation', 'pathology', 'Perform', 'Physical', 'place of residence',
              'plication', 'Positive', 'positive', 'Potassium+', 'prescribed', 'Presence', 'presence',
              'presences', 'Present', 'present', 'preserved', 'Pressure', 'pressure', 'pressures', 
              'Prevention', 'prevention', 'Prophylactic', 'Prosthesis', 'prosthesis', 'Protein', 'psych',
              'Reapplication', 'red blood cells', 'Referred', 'Reirradiation', 'Regional', 'Registered', 
              'Release', 'release', 'removal', 'Repair', 'repair', 'repairs', 'report',  'Report', 'reports',
              'resistance index', 'Result',
              'seizures', 'setting', 'Severe', 'severe', 'sharp pain', 'shortening', 'Single', 'single', 
              'Smoking', 'spells', 
              'stabilize', 'stage', 'Surgical', 'stabilized', 'study', 'surgery', 'surgical intervention', 
              'Susceptible', 'swollen', 'Symptomatic', 'syndrome',
              'Therapeutic', 'Therapy', 'therapy', 'Thoracic', 'thr', 'Thrombocytes', 'Tolerance', 'tolerance', 
              'tolerances', 'Treatment', 'treatment',
              'ultrasound', 'ultrasounds', 'Undifferentiated',
              'W exercise', 'weakness', 'weight',
              'year']

In [ ]:
from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import os

models = os.listdir('ner_results_2ver') #в ver1 не включаем модель 1, она не подходит
models

['ner_pretrained_m9.json',
 'ner_pretrained_m9_2.json',
 'ner_pretrained_m8_2.json',
 'ner_pretrained_m8.json',
 'ner_pretrained_m3.json',
 'ner_pretrained_m3_2.json',
 'ner_pretrained_m4.json',
 'ner_pretrained_m4_2.json',
 'ner_pretrained_m5.json',
 'ner_pretrained_m5_2.json',
 'ner_pretrained_m6.json',
 'ner_pretrained_m6_2.json',
 'ner_pretrained_m7_2.json',
 'ner_pretrained_m7.json',
 'filtered']

In [ ]:
with open('ner_results_2ver/ner_pretrained_m9_2.json', 'r') as f:
  terms = json.load(f)

In [ ]:
#после оценки ner был выбран следующий файл (как финальный результат по ner)
with open('umls_m3_2_terms.json', 'r') as f:
  terms = json.load(f)

In [ ]:
terms['1'][:5]

['anaplastic astrocytoma',
 'tumor',
 'hypertension',
 'thigh numbness',
 'right anterior thigh numbness']

Additional filtering

In [ ]:
def first_filter(terms):
  terms = [i for i in terms if i not in stop_words or i not in stopwords.words('english')]
  terms = [i for i in terms if len(i) > 3]
  return terms

In [ ]:
# уберем повторы, пункцуацию и стоп-слова
import string
exclude = set(string.punctuation)

new_terms = {}
for key in terms.keys():
  new_terms[key] = list(set(terms[key]))
  new_terms[key] = [i for i in new_terms[key] if i not in exclude]
  new_terms[key] = first_filter(new_terms[key])

In [ ]:
terms = new_terms

In [ ]:
terms['1'][:5]

['difficulty walking',
 'unresectable anaplastic astrocytoma',
 'field radiation t',
 'right anterior thigh numbness',
 'hypertension']

In [ ]:
def mean_len(terms_list):
  all_lens = [len(term) for term in terms_list]
  res = 0 if len(all_lens) == 0 else (float(sum(all_lens)) / len(all_lens))

  return res

In [ ]:
mean_len(terms['1'])

19.210526315789473

In [ ]:
def filter_terms(terms):
  mean_lens = mean_len(terms)
  threshold = int(mean_lens/2)

  new_terms = []
  for i in terms:
    if len(i) >= threshold and len(i) > 3:
      new_terms.append(i)
  
  return new_terms

In [ ]:
filter_terms(terms['1'])[:5]

['difficulty walking',
 'unresectable anaplastic astrocytoma',
 'field radiation t',
 'right anterior thigh numbness',
 'hypertension']

In [ ]:
terms_filtered = {}
for i in terms.keys():
  terms_filtered[i] = filter_terms(terms[i])

In [ ]:
for i in terms.keys():
  print(len(terms[i]), len(terms_filtered[i]))
  break

19 17


In [ ]:
with open('ner_results_2ver/filtered/ner_m9_2.json', 'w') as f:
  json.dump(terms, f)

with open('ner_results_2ver/filtered/ner_m9_2_filtered.json', 'w') as f:
  json.dump(terms_filtered, f)

In [ ]:
with open('umls_m3_2_terms_f1.json', 'w') as f:
  json.dump(terms, f)

with open('umls_m3_2_terms_f1_filtered.json', 'w') as f:
  json.dump(terms_filtered, f)

In [ ]:
def add_filter(terms): #for model 9
  all_lens = [len(term) for term in terms]
  threshold = int(max(all_lens)*0.8)

  new_terms = []
  for i in terms:
    if len(i) >= threshold:
      new_terms.append(i)
  
  return new_terms

In [ ]:
add_filter(terms['1'])[:5]

['complicated', 'hypertension', 'anaplastic', 'presented', 'extremity']

In [ ]:
terms_add = {}
for i in terms.keys():
  terms_add[i] = add_filter(terms[i])

terms_filtered_add = {}
for i in terms_filtered.keys():
  terms_filtered_add[i] = add_filter(terms_filtered[i])

In [ ]:
for i in terms.keys():
  print(len(terms_add[i]), len(terms_filtered_add[i]))
  break

12 12


In [ ]:
with open('ner_results_2ver/filtered/ner_m9_2_add.json', 'w') as f:
  json.dump(terms_add, f)

with open('ner_results_2ver/filtered/ner_m9_2_filtered_add.json', 'w') as f:
  json.dump(terms_filtered_add, f)

In [ ]:
terms = terms_add
terms_filtered = terms_filtered_add

Load papers

In [ ]:
import time

def get_papers(terms):
  all_recs = {}

  for i in terms.keys():
    #print(f'processing {i}, len = {len(terms[i])}')
    recs_i = []
    for term in terms[i]:
      time.sleep(0.6)
      term_results = medlineplus_search_by_term(term)
      recs_i.extend(term_results)

    all_recs[i] = recs_i

  return all_recs   

In [ ]:
recs = get_papers(terms)

In [ ]:
recs_filtered = get_papers(terms_filtered)

In [ ]:
len(recs['1']), len(recs_filtered['1'])

(21, 18)

In [ ]:
def set_default(obj):
  if isinstance(obj, set):
    return list(obj)
  raise TypeError

!9_2 with add_filter

In [ ]:
with open('models_recs_2ver/texts_recs_m9_2_per3.json', 'w') as f:
  json.dump(recs, f, default=set_default)

In [ ]:
with open('models_recs_2ver/texts_recs_m9_2_filtered_per3.json', 'w') as f:
  json.dump(recs_filtered, f, default=set_default)

In [ ]:
with open('texts_recs_umls_m3_2.json', 'w') as f:
  json.dump(recs, f, default=set_default)

In [ ]:
with open('texts_recs_umls_m3_2_filtered.json', 'w') as f:
  json.dump(recs_filtered, f, default=set_default)